In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
from dateutil import parser
import matplotlib.pyplot as plt
import calendar
from datetime import timedelta

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler

E:\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
data_df = pd.read_csv('D:/BlueDart/POC/Data_v5.csv')
data_df.head()

FileNotFoundError: File b'D:/BlueDart/POC/Data_v5.csv' does not exist

In [3]:
data_df.columns

Index(['Run Type', 'NCARNETNO', 'DFLIGHTDT', 'CFLIGHTNO', 'Route', 'CORG',
       'O_City_type', 'Org-Reg', 'CDST', 'D_City_type',
       ...
       'Weekday_Sunday', 'Weekday_Thursday', 'Weekday_Tuesday',
       'Weekday_Wednesday', 'Time_Interval_[0, 4)', 'Time_Interval_[4, 8)',
       'Time_Interval_[8, 12)', 'Time_Interval_[12, 16)',
       'Time_Interval_[16, 20)', 'Time_Interval_[20, 24)'],
      dtype='object', length=180)

In [4]:
# cont_features = ['Distance','Capacity','Pcs','Departure delay']
cont_features = ['Distance','Capacity','Pcs']
scaler = StandardScaler()
scaled_features = pd.DataFrame(scaler.fit_transform(data_df[cont_features]), columns = cont_features)

In [5]:
data_df.drop(cont_features, axis = 1, inplace=True)
data_df.shape

(90076, 177)

In [6]:
data_df = pd.concat([data_df, scaled_features], axis = 1)
data_df.shape

(90076, 180)

In [7]:
data = data_df.loc[(data_df['Delay']>2)| (data_df['Delay']<-2),:]
# data = data_df.copy()
data = data.reset_index(drop=True)
data.shape

(39304, 180)

In [8]:
# data = data_df.copy()

In [9]:
cat_features = ['Run Type_AIRNET', 'Run Type_SFC', 'Run Type_SPEED', 'TYPE_MXL', 'TYPE_SXL',
                'Crit_40-50%', 'Crit_50-60%', 'Crit_60-70%', 'Crit_<40%', 'Crit_>70%', 
                'Intra/Inter_Inter',
                'CCONTTYPE_B', 'CCONTTYPE_C','Sch/NSch/Spl_Sch Run', 'Sch/NSch/Spl_Spl Run', 'O_D_EAST-EAST',
                'O_D_EAST-NORTH', 'O_D_EAST-SOUTH1','O_D_EAST-SOUTH2', 'O_D_EAST-WEST1', 'O_D_EAST-WEST2', 
                'O_D_NORTH-EAST', 'O_D_NORTH-NORTH', 'O_D_NORTH-SOUTH1','O_D_NORTH-SOUTH2',
                'O_D_NORTH-WEST1', 'O_D_NORTH-WEST2', 'O_D_SOUTH1-EAST', 'O_D_SOUTH1-NORTH', 'O_D_SOUTH1-SOUTH1',
                'O_D_SOUTH1-SOUTH2', 'O_D_SOUTH1-WEST1', 'O_D_SOUTH1-WEST2', 'O_D_SOUTH2-EAST', 'O_D_SOUTH2-NORTH', 'O_D_SOUTH2-SOUTH1',
                'O_D_SOUTH2-SOUTH2', 'O_D_SOUTH2-WEST1', 'O_D_SOUTH2-WEST2', 'O_D_WEST1-EAST', 'O_D_WEST1-NORTH', 'O_D_WEST1-SOUTH1',
                'O_D_WEST1-SOUTH2', 'O_D_WEST1-WEST1', 'O_D_WEST1-WEST2', 'O_D_WEST2-EAST', 'O_D_WEST2-NORTH', 'O_D_WEST2-SOUTH1',
                'O_D_WEST2-SOUTH2', 'O_D_WEST2-WEST1', 'O_D_WEST2-WEST2', 'Imp_Routes_ADL', 'Imp_Routes_AVPWBBA', 'Imp_Routes_BAM',
                'Imp_Routes_BBL', 'Imp_Routes_BCO', 'Imp_Routes_BTM', 'Imp_Routes_BTR', 'Imp_Routes_BTS', 'Imp_Routes_COB',
                'Imp_Routes_GBT', 'Imp_Routes_GVH', 'Imp_Routes_GXK', 'Imp_Routes_IBP', 'Imp_Routes_KSG', 'Imp_Routes_LAM', 
                'Imp_Routes_MOB', 'Imp_Routes_NAC', 'Imp_Routes_Others', 'Imp_Routes_PQB', 'Imp_Routes_STC', 'Imp_Routes_TRB',
                'Imp_Vendors_ABC10', 'Imp_Vendors_ABC12', 'Imp_Vendors_ABC124', 'Imp_Vendors_ABC15', 'Imp_Vendors_ABC180',
                'Imp_Vendors_ABC211', 'Imp_Vendors_ABC251', 'Imp_Vendors_ABC27', 'Imp_Vendors_ABC31', 'Imp_Vendors_ABC34', 
                'Imp_Vendors_ABC36', 'Imp_Vendors_ABC49', 'Imp_Vendors_ABC51', 'Imp_Vendors_ABC54', 'Imp_Vendors_ABC57', 
                'Imp_Vendors_ABC58', 'Imp_Vendors_ABC60', 'Imp_Vendors_ABC69', 'Imp_Vendors_ABC78', 'Imp_Vendors_ABC85', 
                'Imp_Vendors_ABC9', 'Imp_Vendors_ABC97', 'Imp_Vendors_Others', 'Imp_CVEHTYPE_H20', 'Imp_CVEHTYPE_H24',
                'Imp_CVEHTYPE_L10', 'Imp_CVEHTYPE_L14', 'Imp_CVEHTYPE_L17', 'Imp_CVEHTYPE_LCV', 'Imp_CVEHTYPE_M17',
                'Imp_CVEHTYPE_M18', 'Imp_CVEHTYPE_M19', 'Imp_CVEHTYPE_M20', 'Imp_CVEHTYPE_MXL', 'Imp_CVEHTYPE_Others',
                'Imp_CVEHTYPE_VAN', 'O_D_city_type_Metro/Metro', 'O_D_city_type_Metro/Non-Metro', 'O_D_city_type_Non-Metro/Metro',
                'O_D_city_type_Non-Metro/Non-Metro', 'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7',
                'Month_8', 'Weekday_Friday', 'Weekday_Monday', 'Weekday_Saturday', 'Weekday_Sunday', 'Weekday_Thursday', 'Weekday_Tuesday',
                'Weekday_Wednesday', 'Time_Interval_[0, 4)','Time_Interval_[4, 8)', 'Time_Interval_[8, 12)',
                'Time_Interval_[12, 16)', 'Time_Interval_[16, 20)', 'Time_Interval_[20, 24)']
len(cat_features)

133

In [10]:
column_list = cat_features + cont_features

X = data[column_list]
y = data['Delay_Flag']

In [11]:
X.shape

(39304, 136)

In [12]:
rf = RandomForestClassifier(n_estimators=50, max_depth = 6, random_state = 0)
rf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=6, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
print(classification_report(y, rf.predict(X)))

             precision    recall  f1-score   support

          0       0.90      0.27      0.42     16204
          1       0.66      0.98      0.79     23100

avg / total       0.76      0.69      0.63     39304



In [14]:
columns = list(pd.Series(rf.feature_importances_, X.columns).sort_values(ascending = False)[:20].index)
print(pd.Series(rf.feature_importances_, X.columns).sort_values(ascending = False)[:20])

Imp_Vendors_ABC10                0.124833
Imp_Routes_Others                0.100034
Distance                         0.064843
Imp_Vendors_ABC15                0.052752
Run Type_SPEED                   0.040343
O_D_SOUTH2-SOUTH2                0.037889
Imp_Vendors_ABC60                0.034508
Run Type_SFC                     0.031967
Intra/Inter_Inter                0.023544
O_D_city_type_Non-Metro/Metro    0.023324
O_D_WEST1-SOUTH2                 0.022919
Sch/NSch/Spl_Sch Run             0.022041
Time_Interval_[20, 24)           0.021403
Imp_Vendors_ABC85                0.017875
Capacity                         0.017609
Imp_Routes_BCO                   0.016652
Imp_Routes_STC                   0.016174
Imp_Routes_BBL                   0.015441
O_D_city_type_Metro/Non-Metro    0.015318
O_D_NORTH-SOUTH1                 0.015289
dtype: float64


In [15]:
# columns.remove('Intra/Inter_Inter')
# columns.remove('Sch/NSch/Spl_Sch Run')
# columns.remove('Run Type_SFC')
# columns.remove('Imp_CVEHTYPE_H24')
# columns.remove('Capacity')
# columns.remove('O_D_city_type_Non-Metro/Metro')
len(columns)

20

In [16]:
X = X[columns]
X.shape

(39304, 20)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0,stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((27512, 20), (11792, 20), (27512,), (11792,))

In [18]:
sum(y_test)/len(y_test), sum(y_train)/len(y_train)

(0.5876865671641791, 0.5877435300959581)

In [20]:
model_name = '12102018/Model_ND2_D2_20Var_Standard_noDD'

In [21]:
model = sm.Logit(y_train, X_train, random_state = 1)
result = model.fit(maxiter = 100)
result.summary()

Optimization terminated successfully.
         Current function value: 0.578196
         Iterations 9


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Delay_Flag   No. Observations:                27512
Model:                          Logit   Df Residuals:                    27492
Method:                           MLE   Df Model:                           19
Date:                Thu, 13 Dec 2018   Pseudo R-squ.:                  0.1468
Time:                        11:48:19   Log-Likelihood:                -15907.
converged:                       True   LL-Null:                       -18644.
                                        LLR p-value:                     0.000
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Imp_Vendors_ABC10                -1.8549      0.090    -20.497      0.000      -2.032      -1.678
Imp_Routes_Others                 1.4225      0.064     22.268      0.000       1.297       1.548
Distance                         -0.1659      0.020     -8.119      0.000      -0.206      -0.126
Imp_Vendors_ABC15                -2.7222      0.151    -18.041      0.000      -3.018      -2.426
Run Type_SPEED                    0.6914      0.083      8.358      0.000       0.529       0.853
O_D_SOUTH2-SOUTH2                -0.4262      0.065     -6.559      0.000      -0.554      -0.299
Imp_Vendors_ABC60                 3.5495      0.338     10.505      0.000       2.887       4.212
Run Type_SFC                     -0.4055      0.067     -6.063      0.000      -0.537      -0.274
Intra/Inter_Inter                -0.1933      0.045     -4.252      0.000      -0.282      -0.104
O_D_city_type_Non-Metro/Metro    -0.2031      0.035     -5.826      0.000      -0.271      -0.135
O_D_WEST1-SOUTH2                  1.0827      0.111      9.776      0.000       0.866       1.300
Sch/NSch/Spl_Sch Run             -0.5032      0.030    -16.987      0.000      -0.561      -0.445
Time_Interval_[20, 24)           -0.5364      0.037    -14.434      0.000      -0.609      -0.464
Imp_Vendors_ABC85                -5.3406      1.005     -5.315      0.000      -7.310      -3.371
Capacity                          0.1380      0.020      7.041      0.000       0.100       0.176
Imp_Routes_BCO                   -1.5735      0.211     -7.443      0.000      -1.988      -1.159
Imp_Routes_STC                   -1.3637      0.259     -5.272      0.000      -1.871      -0.857
Imp_Routes_BBL                    3.4370      0.344      9.980      0.000       2.762       4.112
O_D_city_type_Metro/Non-Metro     0.3550      0.035     10.279      0.000       0.287       0.423
O_D_NORTH-SOUTH1                 -0.8709      0.072    -12.074      0.000      -1.012      -0.730
=================================================================================================
"""

In [22]:
while (max(result.pvalues[pd.notnull(result.pvalues)]) > 0.05):
    col_rem = list(result.pvalues[result.pvalues == max(result.pvalues[pd.notnull(result.pvalues)])].index)
    print('Column Removed:',col_rem)
    cols = [c for c in X_train.columns if c not in col_rem]
    X_train = X_train[cols]
    model = sm.Logit(y_train, X_train, random_state = 1)
    result = model.fit(maxiter = 100)

In [23]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             Delay_Flag   No. Observations:                27512
Model:                          Logit   Df Residuals:                    27492
Method:                           MLE   Df Model:                           19
Date:                Thu, 13 Dec 2018   Pseudo R-squ.:                  0.1468
Time:                        11:48:20   Log-Likelihood:                -15907.
converged:                       True   LL-Null:                       -18644.
                                        LLR p-value:                     0.000
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Imp_Vendors_ABC10                -1.8549      0.090    -20.497      0.000      -2.032      -1.678
Imp_Routes_Others                 1.4225      0.064     22.268      0.000       1.297       1.548
Distance                         -0.1659      0.020     -8.119      0.000      -0.206      -0.126
Imp_Vendors_ABC15                -2.7222      0.151    -18.041      0.000      -3.018      -2.426
Run Type_SPEED                    0.6914      0.083      8.358      0.000       0.529       0.853
O_D_SOUTH2-SOUTH2                -0.4262      0.065     -6.559      0.000      -0.554      -0.299
Imp_Vendors_ABC60                 3.5495      0.338     10.505      0.000       2.887       4.212
Run Type_SFC                     -0.4055      0.067     -6.063      0.000      -0.537      -0.274
Intra/Inter_Inter                -0.1933      0.045     -4.252      0.000      -0.282      -0.104
O_D_city_type_Non-Metro/Metro    -0.2031      0.035     -5.826      0.000      -0.271      -0.135
O_D_WEST1-SOUTH2                  1.0827      0.111      9.776      0.000       0.866       1.300
Sch/NSch/Spl_Sch Run             -0.5032      0.030    -16.987      0.000      -0.561      -0.445
Time_Interval_[20, 24)           -0.5364      0.037    -14.434      0.000      -0.609      -0.464
Imp_Vendors_ABC85                -5.3406      1.005     -5.315      0.000      -7.310      -3.371
Capacity                          0.1380      0.020      7.041      0.000       0.100       0.176
Imp_Routes_BCO                   -1.5735      0.211     -7.443      0.000      -1.988      -1.159
Imp_Routes_STC                   -1.3637      0.259     -5.272      0.000      -1.871      -0.857
Imp_Routes_BBL                    3.4370      0.344      9.980      0.000       2.762       4.112
O_D_city_type_Metro/Non-Metro     0.3550      0.035     10.279      0.000       0.287       0.423
O_D_NORTH-SOUTH1                 -0.8709      0.072    -12.074      0.000      -1.012      -0.730
=================================================================================================
"""

In [24]:
logreg = LogisticRegression(random_state = 1)
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [25]:
coeff_df = pd.DataFrame({'Feature' : X_train.columns,'Beta_Coeff':logreg.coef_[0]})
coeff_df = coeff_df.append({'Feature':'Intercept','Beta_Coeff':logreg.intercept_[0]}, ignore_index=True)
coeff_df.to_csv('D:/BlueDart/POC/Model_Iterations/Final Model/'+model_name + '_coeff.csv', index = False)

In [27]:
coeff_df

,Beta_Coeff,Feature
0,-1.816401,Imp_Vendors_ABC10
1,1.526639,Imp_Routes_Others
2,-0.175120,Distance
3,-2.680985,Imp_Vendors_ABC15
4,1.589932,Run Type_SPEED
5,-0.417442,O_D_SOUTH2-SOUTH2
6,3.239647,Imp_Vendors_ABC60
7,0.488235,Run Type_SFC
8,-0.177699,Intra/Inter_Inter
9,-0.193600,O_D_city_type_Non-Metro/Metro


In [28]:
y_pred_train = logreg.predict(X_train)
print('Accuracy of logistic regression classifier on train set: {:.4f}'.format(logreg.score(X_train, y_train)))

Accuracy of logistic regression classifier on train set: 0.6894


In [29]:
y_pred_test = logreg.predict(X_test[X_train.columns])
print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(logreg.score(X_test[X_train.columns], y_test)))

Accuracy of logistic regression classifier on test set: 0.6831


In [37]:
for cutoff in [0.3,0.35,0.4,0.45,0.5,0.55]:
    
    y_pred_train = [int(i[1]) for i in logreg.predict_proba(X_train[X_train.columns])>cutoff]
    print(cutoff)
    print(precision_score(y_train, y_pred_train), recall_score(y_train, y_pred_train))
    

0.3
0.6487290762554246 0.9706864564007421
0.35
0.6535078489962544 0.9602968460111317
0.4
0.6552198392573172 0.942795299938157
0.45
0.6684104627766599 0.9244897959183673
0.5
0.6848637641811306 0.8735930735930736
0.55
0.7054537301720756 0.7783549783549784


In [38]:
y_pred_train = [int(i[1]) for i in logreg.predict_proba(X_train[X_train.columns])>0.5]
y_pred_test = [int(i[1]) for i in logreg.predict_proba(X_test[X_train.columns])>0.5]
print(precision_score(y_train, y_pred_train), recall_score(y_train, y_pred_train), accuracy_score(y_train, y_pred_train))
print(precision_score(y_test, y_pred_test), recall_score(y_test, y_pred_test), accuracy_score(y_test, y_pred_test))

0.6848637641811306 0.8735930735930736 0.6894446059901134
0.6807426695346994 0.8676767676767677 0.6830902306648575


In [39]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_train, y_pred_train)
pd.DataFrame(confusion_matrix)

,0,1
0,4842,6500
1,2044,14126


In [43]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred_test)
pd.DataFrame(confusion_matrix)

,0,1
0,2042,2820
1,917,6013


In [45]:
def create_ks_table(data,Score_Column,Target):

    #deciles = pd.qcut(data[Score_Column], 10)
    data['score_dec_bucket'] = pd.qcut(data[Score_Column], 10)
    df = data.groupby('score_dec_bucket',as_index=True).agg({Target: ['sum','count']})
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    df['cum_sum_'+Target] = np.cumsum(df['Target_sum'])
    df['cum_sum_'+Target+'Perc'] = df['cum_sum_'+Target]/df['Target_sum'].sum()
    df = df.reset_index()
    x = list(reversed(np.cumsum([i for i in reversed(df['Target_sum'])])))
    df['reversed_cumsum'] = [i for i in x]
    deciles = df['score_dec_bucket'].values
    deciles = list(map(lambda x: eval(str(x).replace('(', '[')), deciles))

    buckets = np.array([0, deciles[-3][0],  deciles[-1][0], 1])

    return df, buckets


In [46]:
y_train_prob_all = logreg.predict_proba(X_train[X_train.columns])
y_train_prob = [i[1] for i in y_train_prob_all]

y_train_df = pd.DataFrame({'Score':y_train_prob,'Target':y_train})
ks_table_outsample, _ = create_ks_table(y_train_df,'Score','Target')
ks_table_outsample.to_csv('D:/BlueDart/POC/Model_Iterations/Final Model/'+model_name + '_KS_Train.csv', index = False)
ks_table_outsample

,score_dec_bucket,Target_sum,Target_count,cum_sum_Target,cum_sum_TargetPerc,reversed_cumsum
0,"(0.0259, 0.253]",328,2757,328,0.020284,16170
1,"(0.253, 0.467]",991,2778,1319,0.081571,15842
2,"(0.467, 0.525]",1612,2773,2931,0.181262,14851
3,"(0.525, 0.575]",1401,2700,4332,0.267904,13239
4,"(0.575, 0.621]",1722,2771,6054,0.374397,11838
5,"(0.621, 0.664]",1530,2737,7584,0.469017,10116
6,"(0.664, 0.711]",1825,2742,9409,0.581880,8586
7,"(0.711, 0.749]",2080,2751,11489,0.710513,6761
8,"(0.749, 0.806]",2255,2757,13744,0.849969,4681
9,"(0.806, 0.987]",2426,2746,16170,1.000000,2426


In [47]:
y_test_prob_all = logreg.predict_proba(X_test[X_train.columns])
y_test_prob = [i[1] for i in y_test_prob_all]

y_test_df = pd.DataFrame({'Score':y_test_prob,'Target':y_test})
ks_table_outsample, _ = create_ks_table(y_test_df,'Score','Target')
ks_table_outsample.to_csv('D:/BlueDart/POC/Model_Iterations/Final Model/'+model_name + '_KS_Test.csv', index = False)
ks_table_outsample

,score_dec_bucket,Target_sum,Target_count,cum_sum_Target,cum_sum_TargetPerc,reversed_cumsum
0,"(0.0259, 0.251]",143,1180,143,0.020635,6930
1,"(0.251, 0.467]",430,1187,573,0.082684,6787
2,"(0.467, 0.525]",707,1192,1280,0.184704,6357
3,"(0.525, 0.574]",590,1159,1870,0.269841,5650
4,"(0.574, 0.621]",734,1183,2604,0.375758,5060
5,"(0.621, 0.664]",655,1180,3259,0.470274,4326
6,"(0.664, 0.711]",774,1181,4033,0.581962,3671
7,"(0.711, 0.75]",905,1175,4938,0.712554,2897
8,"(0.75, 0.807]",964,1175,5902,0.851659,1992
9,"(0.807, 0.987]",1028,1180,6930,1.000000,1028


In [48]:
data1 = data_df.loc[(data_df['Delay']<=2) & (data_df['Delay']>=-2),:]
data1.shape

(50772, 180)

In [50]:
data2 = pd.concat([data1[X_train.columns], X_test[X_train.columns]])
data2.shape

(62564, 20)

In [51]:
y_data2 = pd.concat([data1['Delay_Flag'], y_test])
y_data2.shape

(62564,)

In [52]:
sum(y_data2 == 0), sum(y_data2)

(40132, 22432)

In [53]:
y1 = logreg.predict(data2)
sum(y1), sum(y1 == 0)

(45649, 16915)

In [1]:
from sklearn.metrics import confusion_matrix
confusion_matrix_1 = confusion_matrix(y_data2, y1)
pd.DataFrame(confusion_matrix_1)

NameError: name 'y_data2' is not defined

In [60]:
print(classification_report(y_data2, y1))

             precision    recall  f1-score   support

          0       0.75      0.32      0.44     40132
          1       0.40      0.81      0.53     22432

avg / total       0.62      0.49      0.48     62564



In [61]:
y1_prob_all = logreg.predict_proba(data1[X_train.columns])
y1_prob = [i[1] for i in y1_prob_all]
len(y1_prob)

50772

In [63]:
# y1_df = pd.DataFrame({'Score':y1_prob,'Target':y1})
# ks_table_outsample, _ = create_ks_table(y1_df,'Score','Target')
# ks_table_outsample.to_csv('D:/BlueDart/POC/Model_Iterations/Final Model/'+model_name + '_KS_Others_equal2.csv', index = False)
# ks_table_outsample